In [2]:
!unzip \*.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [19]:
import pandas as pd
data = pd.read_csv("train.csv")
data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [20]:
# Vemos las variables
features = data.columns.values
features

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [21]:
# Info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [22]:
# Vemos que columnas tienen valores vacios y cuantos
dict_nuls = {}
for col in features:
  if data[col].isnull().any():
    dict_nuls[col] = data[col].isnull().sum()
dict_nuls

{'Age': 177, 'Cabin': 687, 'Embarked': 2}

In [23]:
# Vemos cuantos valores distintos hay en ticket
data['Ticket'].nunique() #681, 70% son distintos
# Eliminamos
del data['Ticket']
# Eliminamos embarked oporuqe no deberia influir hacia donde iba
del data['Embarked']

In [24]:
# El nombre no deberia influir (igual en vola si es cuico sobrevive jajaj)
del data['Name']

# El sexo se debe hacer dummie
# La edad llenar con el promedio
# CAbina llenamos con None y dummie


In [26]:
del data['PassengerId']
X = data.iloc[:,1:]
y = data.iloc[:,0]


GRAFICA PO MADAFACA

In [ ]:
import matplotlib.pyplot as plt

In [31]:
categorical = ['Sex' 'Cabin']
numerical = ['Pclass','SibSp','Parch','Fare']

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder()

X_train, X_test, y_train, y_test = train_test_split(X,y)

cat_pipe = Pipeline([
                     ('imputer', SimpleImputer(strategy='constant', fill_value = 'missing')),
                     ('dummie', LabelEncoder())
])
num_pipe = Pipeline([
                     ('imputer', SimpleImputer(strategy = 'median'))
])
preprocess = ColumnTransformer([
                                ('cat', categorical, cat_pipe),
                                ('num', numerical, num_pipe)
])

pipe = Pipeline([
                 ('preprocess', preprocess),
                 ('classifier', ???)
])


array(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin'],
      dtype=object)